In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
from my_utils.preprocess import DataPreprocess
from my_utils.nfold_test import NFoldModel
from my_utils.dataloader import Workout_dataset
from my_utils.model import CNN_RNN_Resnet

In [2]:
# seed 고정
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(seed)
np.random.seed(seed)
rn.seed(seed)

In [3]:
features_path = './data/train_features.csv'
labels_path = './data/train_labels.csv'

preprocess = DataPreprocess(
    folds=10,seed=seed,acc_norm=10,gy_norm=2000,
    features_path=features_path,labels_path=labels_path)

In [4]:
preprocess.labels_to_csv()
preprocess.features_to_npy()

In [5]:
nfold_model = NFoldModel(batch_size=64, valid_ratio=4, lr=0.001, early_stop_patience=30, seed=42)
nfold_model.Model_class = CNN_RNN_Resnet
nfold_model.Workout_dataset = Workout_dataset
nfold_model.model_parameter = dict(leakyrelu_alpha = 0.1, 
    input_kernels = 20, input_kernel_width = 3, input_regularize_coeff=0.001,
    res_kernels = 60, res_kernel_width = 3, res_regularize_coeff=0.1, res_num = 5
    )
nfold_model.file_base_name = 'CNN_RNN_Resnet'

In [6]:
fold_list = [1,3,5,7,9]

In [7]:
nfold_model.nfold_train(fold_list)

 improve from 0.98310
Epoch 97/2000
35/35 [==============================] - 15s 438ms/step - loss: 0.7547 - accuracy: 0.9393 - val_loss: 0.9529 - val_accuracy: 0.8789

Epoch 00097: val_loss improved from 0.98310 to 0.95293, saving model to ./save/CNN_RNN_Resnet_fold_5.hdf5
Epoch 98/2000
35/35 [==============================] - 16s 442ms/step - loss: 0.7391 - accuracy: 0.9464 - val_loss: 1.0114 - val_accuracy: 0.8711

Epoch 00098: val_loss did not improve from 0.95293
Epoch 99/2000
35/35 [==============================] - 16s 446ms/step - loss: 0.7454 - accuracy: 0.9460 - val_loss: 1.0021 - val_accuracy: 0.8672

Epoch 00099: val_loss did not improve from 0.95293
Epoch 100/2000
35/35 [==============================] - 15s 439ms/step - loss: 0.7227 - accuracy: 0.9576 - val_loss: 1.0005 - val_accuracy: 0.8672

Epoch 00100: val_loss did not improve from 0.95293
Epoch 101/2000
35/35 [==============================] - 15s 438ms/step - loss: 0.7216 - accuracy: 0.9558 - val_loss: 1.0075 - val_

In [13]:
import time
test_batch_size = 625
for fold in fold_list:
    nfold_model.evaluate_test(test_batch_size,fold)
    time.sleep(2)

1/1 [==============================] - 16s 16s/step - loss: 0.9587 - accuracy: 0.8688


In [14]:
predict = nfold_model.nfold_predict(test_batch_size,fold_list)
score_dict = nfold_model.nfold_score(preprocess.y_test,predict,average='macro')
print(score_dict)

{'accuracy': 0.8816, 'recall': 0.7463033515233688, 'precision': 0.8037728751395452, 'f1': 0.7600673509215357}


In [15]:
predict = nfold_model.nfold_predict(test_batch_size,fold_list)
score_dict = nfold_model.nfold_score(preprocess.y_test,predict,average='weighted')
print(score_dict)

{'accuracy': 0.8816, 'recall': 0.8816, 'precision': 0.8764713725490195, 'f1': 0.8717254745254746}
